### ----------------------------------------------------------------------------------------------------------
# プログラミング(Python) e-Learning用教材 演習編
## 社会変革型 ライフサイエンス・ヘルスケア データサイエンティスト育成講座
### Ver0.2(α版) 2019/1/11 Akira Izumi
### ----------------------------------------------------------------------------------------------------------

## ピマインディアンの糖尿病データ
https://github.com/niharikagulati/diabetesprediction/blob/master/diabetes.csv  
読み込んだデータの変数の意味については以下を参照

| column # | variables |
|:---------|:----------|
| 0 | Number of times pregnant |
| 1 | Plasma glucose concentration a 2 hours in an oral glucose tolerance test [mg/dL] |
| 2 | Diastolic blood pressure [mm/Hg] |
| 3 | Triceps skins fold thickness [mm] |
| 4 | 2-hour serum insulin [mu U/ml] |
| 5 | Body Mass Index |
| 6 | Diabetes pedigree function |
| 7 | Age [years] |
| 8 | Outcome:  Class Variable {0, 1} where '1' denotes patient having diabetes | 

###  注意: データに欠損値があるため、null値をmode（最頻値）やmean（平均値）で置換することを推奨されています
> Data Cleaning will take place as data has got lot of missing values.  
> Handling missing values can be done either by replacing null values with mode or mean or replacing the null value with a random variable.

In [ ]:
# データ読み込み

import pandas as pd

# Jupyter Notebookのホームディレクトリに上記csvファイルを格納してください

# ファイル名
filename = "pima-indians-diabetes.data.csv"

# Pandasのread_csvメソッドを使用し、csvファイルを読み込みます
# オプションのsepは区切り文字を指定します。
# また、読み込むcsvファイルに列名がないため、namesオプションで列名を付与します
df = pd.read_csv(filename, sep=',',
                 names=[
                     'Pregnancies',
                     'Glucose',
                     'Blood Pressure',
                     'Skin Thickness',
                     'Insulin',
                     'BMI',
                     'Diabetes Pedigree Function',
                     'Age',
                     'Outcome'
                 ]
                )

※ファイルのパスを指定してもデータを読み込めます
Linux/Unixであれば問題ありませんが、Windowsのパスを入れる場合は以下の対応が必要になります。
- 文字列の前に"r"を入れる  
例: filepath = r'C:\Users\ntohmatsu\python_data\pima-indians-diabetes.csv'
- 区切り文字の円マークを1文字だけでなく、2文字入れる  
例: filepath = 'C:\\Users\\ntohmatsu\\python_data\\pima-indians-diabetes.csv'

df = pd.read_csv(filepath, sep=',',
                 names=[
                     'Pregnancies',
                     'Glucose',
                     'Blood Pressure',
                     'Skin Thickness',
                     'Insulin',
                     'BMI',
                     'Diabetes Pedigree Function',
                     'Age',
                     'Outcome'
                 ]
                )

In [ ]:
# 読み込んだデータの確認

df.head()

In [ ]:
# 行数・列数の確認

df.shape

In [ ]:
# 各変数のデータ型の確認

df.dtypes

## データの要約・可視化

In [ ]:
# 各列にNullが何個あるか確認します。

# メソッドisnullは、各要素が欠損値(NaN)であればTrue, そうでなければFalseを出力します
# メソッドsumは、値の合計を出力します。オプションのaxis=0で行の合計、axis=1で列の合計を出力します
# また、Trueは1、Falseは0と等価であることに注意してください

# 結果から、欠損値はNaN以外で表現されていることが分かります

print(df.isnull().sum(axis=0))

In [ ]:
# 基本統計量の算出

# データの個数、平均、楊淳偏差、最小値、データを昇順ソートしたときに25%, 50%, 75%目の値、最大値

"""
Pregnancies（妊娠回数）が'0'というのは違和感ありませんが、
Glucose（血糖濃度）やBlood Pressure（血圧）が0というのは違和感があります
そのため、欠損値は0として入力されていることが分かります
"""

df.describe()

In [ ]:
# グループごとの基本統計量の算出

# 糖尿病患者は、非患者と比較して、各変数の値が高い傾向にありそうです
df_group_describe = df.groupby('Outcome').describe()
for col_ind in df.columns[:-1]:
    print(col_ind + ":\n" + str(df_group_describe[col_ind].T) + "\n")

In [ ]:
# データの可視化用ライブラリmatplotlibの利用

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# GlucoseやAgeに差がありそうです
sns.pairplot(df, hue='Outcome')

In [ ]:
# 各データ間の相関を確認

# 画像サイズの設定
plt.figure(figsize=(12, 9))

# 相関係数行列のヒートマップ
# AgeとPregnanciesに比較的高い相関がある等、確認できます
sns.heatmap(df.corr(), cmap='bwr', annot=True)

## 欠損値補完

In [ ]:
# 欠損値の個数を確認

# 0と入力されて不適切な列を列挙
imputer_cols = df.columns[[1,2,3,4,5,6]]

# 各列において、0の個数を確認します
df[imputer_cols][df[imputer_cols]==0].count()

In [ ]:
# 欠損値補完をするため、修正後のデータフレームcorrect_dfを作成します
# 一度、dfと同じデータを読み込みます
correct_df = df.copy()

# 今回は、欠損値0の補完には"算術平均値"（meanメソッド）を用います
# その他、"中央値"（medianアトリビュート）や"最頻値"（modeメソッド）を利用する場合もあります
# 値の置換にはmaskメソッドを利用し、第1引数は条件、第2引数は置換する値になります
for i in imputer_cols:
    correct_df[i] = correct_df[i].mask(df[i]==0, df[i].mean())

# Insulin列の0～2行目を見ると、元々"0"だったものが"79.799479"に置換されていることが分かります
correct_df.head()

In [ ]:
# 欠損値修正後の基本統計量の算出

# データの個数、平均、楊淳偏差、最小値、データを昇順ソートしたときに25%, 50%, 75%目の値、最大値
# 修正対象列の最小値を見ると、0より大きくなっており、欠損値が保管されていることが分かります
correct_df.describe()

In [ ]:
# 欠損値修正後のグループ毎基本統計量の算出

correct_df_group_describe = correct_df.groupby('Outcome').describe()
for col_ind in correct_df.columns[:-1]:
    print(col_ind + ":\n" + str(correct_df_group_describe[col_ind].T) + "\n")

In [ ]:
# データの可視化用ライブラリmatplotlibの利用

sns.pairplot(correct_df, hue='Outcome')

In [ ]:
# 各データ間の相関を確認

# 画像サイズの設定
plt.figure(figsize=(12, 9))

# 相関係数行列のヒートマップ
# AgeとPregnanciesに比較的高い相関がある等、確認できます
sns.heatmap(correct_df.corr(), cmap='bwr', annot=True)

## 機械学習モデルの作成

In [ ]:
# 説明変数と目的変数の選択

# 説明変数の選択
X = correct_df.loc[:, correct_df.columns[:-1]]

# 目的変数の選択
y = correct_df.loc[:, 'Outcome']

# 正しく分割されたことを確認
print("X=\n" + str(X.head()) + '\n')
print("y=\n" + str(y.head()))

In [ ]:
# 学習用データと評価用データの分割

# scikit-learnライブラリに簡単にデータを分割してくれるmodel_selectionメソッドがあります
from sklearn import model_selection

# train_test_splitメソッドを使うことで、
# test_size（下記の場合は33%、つまり学習用データは67%）に従ってデータが分割されます
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=1/5, random_state=0
)

In [ ]:
# データが正しく分割できているか確認

print("学習用説明変数のサイズは{}です".format(X_train.shape))
print("学習用目的変数のサイズは{}です".format(y_train.shape))
print("評価用説明変数のサイズは{}です".format(X_test.shape))
print("評価用目的変数のサイズは{}です".format(y_test.shape))

In [ ]:
# データの標準化

# scikit-learnライブラリに簡単に標準化してくれるStandardScalerクラスが含まれています
from sklearn.preprocessing import StandardScaler

# インスタンスの生成
scaler = StandardScaler()

# 学習用データの標準化
# fit_transformメソッドは各列の平均と分散を用いて標準化します
# DataConversionWarningができますが、int64型をfloat64型に変換するためであり、無視して問題ございません
st_X_train = scaler.fit_transform(X_train)

# 評価用データの標準化
# transformメソッドは、最後にfit_transformを利用したときの平均と分散を用いて標準化します
# こちらもWarningがでますが、先ほどと同様のものであるため、無視して問題ございません。
st_X_test = scaler.transform(X_test)

In [ ]:
#標準化できているか確認
pd.DataFrame(st_X_train, columns=X.columns).describe()

In [ ]:
# ロジスティック回帰モデル

# LogisticRegressionクラスのインポート
from sklearn.linear_model import LogisticRegression

# インスタンスを生成します
# オプションのsolverは最適化手法、max_iterは最大反復回数を表します
lr = LogisticRegression(solver='lbfgs', class_weight='balanced',
                        max_iter=1e5, random_state=0)

# 学習（パラメータの推定）
lr.fit(st_X_train, y_train)

In [ ]:
# 回帰係数の可視化

# DataFrameに格納
coefs = pd.DataFrame(
    lr.coef_,
    columns=X.columns
)
coefs['intercept'] = lr.intercept_

# DataFrameの可視化
coefs.T.plot(y=coefs.index, kind='bar', title='coefficients', grid=True, figsize=(12,9))

In [ ]:
# 予測値の算出

# 既に学習したmodelインスタンスに、predictメソッドを使うことで予測値を出力します
y_pred = lr.predict(st_X_test)

### モデルの精度評価

In [ ]:
# 精度評価の算出（一部可視化）

# 混合行列、classification_reportの確認
from sklearn.metrics import confusion_matrix, classification_report

labels = [1,0]

# classification_reportの出力
print(classification_report(y_test, y_pred, labels=labels))

# 混合行列を出力します
# 医療検査で1を陽性、0を陰性とすると、左上は真陽性、右下は真陰性、
# 右上は偽陽性、左下は偽陰性になります。
conf_mtx = confusion_matrix(y_test, y_pred, labels=labels)
# ヒートマップとして可視化します
# オプションのannotは、ヒートマップの各セルにおける該当数を表示します
sns.heatmap(conf_mtx, annot=True, xticklabels=labels, yticklabels=labels)

### ハイパーパラメータチューニング

In [ ]:
from sklearn.model_selection import GridSearchCV

# 予測モデルの選択
lr_gs = LogisticRegression(class_weight='balanced', max_iter=1e5,
                           random_state=0, dual=False)

# ハイパーパラメータの調整範囲
# 正則化指標'penalty': 'L1ノルム'か'L2ノルム'
# 正則化の強度'C': 10の-5乗～5乗まで、0.5乗刻み
import numpy as np
tuning_params = [
    {
        'penalty': ['l1'],
        'solver': ['saga'],
        'C': 10.**np.arange(-5, 5.5, 0.5)
    },
    {
        'penalty': ['l2'],
        'solver': ['lbfgs'],
        'C': 10.**np.arange(-5, 5.5, 0.5)
    }
]

# グリッドサーチのgspmsインスタンス生成
# 予測モデル: lr_gs（線形サポートベクターマシン識別機）
# 調整するパラメータ: tuning_params
# 交差確認法における分割ブロック数: cv
# スコアリング方法: f1値、マイクロ平均
# 並列処理数: n_jobs（1なら並列処理なし、-1なら全てのプロセッサ使用）
# テストデータのサンプル数に基づくスコアへの重み付け: iid（将来、このオプションは削除される）
# 学習データのスコア出力: return_train_score（現状のdefaultは'warm'だが、将来このオプションはFalseになる）
lr_gs = GridSearchCV(
    lr_gs, tuning_params,
    cv=10, scoring='f1_micro', n_jobs=1, 
    iid=False, return_train_score=False
)

In [ ]:
# グリッドサーチの実施

lr_gs.fit(st_X_train, y_train)

In [ ]:
# 交差検証の結果（cv_results_属性）をDataFrameとして表示
# 平均精度を降順に並べ替え

lr_gs_results = pd.DataFrame(lr_gs.cv_results_)
lr_gs_results.sort_values('mean_test_score', ascending=False)

In [ ]:
print('lr_gs_resultsのサイズ: ' + str(lr_gs_results.shape) )
print('最良平均精度: ' + str(lr_gs_results.loc[lr_gs_results.index[0], 'mean_test_score']))

In [ ]:
# 最適パラメータの確認（best_estimator_属性）

# 交差検証の結果、正則化指標'penalty'はL2ノルム'、
# 正則化の強度'C'は10の-1.5乗の組み合わせが最適であるという結果得る

lr_gs.best_estimator_

In [ ]:
# 判別超平面の係数の可視化

# DataFrameに格納
coefs = pd.DataFrame(
    lr_gs.best_estimator_.coef_,
    columns=df.columns[:-1]
)
coefs['intercept'] = lr_gs.best_estimator_.intercept_

# DataFrameの可視化
coefs.T.plot(y=coefs.index, kind='bar', title='coefficients', grid=True, figsize=(12,9))

In [ ]:
# 予測値の取得

y_pred = lr_gs.predict(st_X_test)

In [ ]:
# 評価指標の出力

# classification_reportの出力
# 結果として、標準的なパラメータと精度はほぼ変わらない
print(classification_report(y_test, y_pred, labels=labels))

# 混同行列の可視化
conf_mtx = confusion_matrix(y_test, y_pred, labels=labels)
sns.heatmap(conf_mtx, annot=True, xticklabels=labels, yticklabels=labels)

### 他モデル（線形サポートベクターマシン識別器）の適用

In [ ]:
# 予測モデルの選択
from sklearn.svm import LinearSVC
lsvc_gs = LinearSVC(class_weight='balanced', tol=1e-3,
                    max_iter=1e4, random_state=0)

# ハイパーパラメータの調整範囲
# 正則化指標'penalty': 'L1ノルム'か'L2ノルム'
# 正則化の強度'C': 10の-5乗～5乗まで、0.5乗刻み
# 損失関数'loss': 'ヒンジ損失'か'二乗ヒンジ損失'

tuning_params = [
    {
        'penalty': ['l2'],
        'loss': ['hinge'],
        'dual': [True],
        'C': 10.**np.arange(-5, 5.5, 0.5)
    },
    {
        'penalty': ['l1'],
        'loss': ['squared_hinge'],
        'dual': [False],
        'C': 10.**np.arange(-5, 5.5, 0.5)
    },
    {
        'penalty': ['l2'],
        'loss': ['squared_hinge'],
        'dual': [True],
        'C': 10.**np.arange(-5, 5.5, 0.5)
    }
]

# グリッドサーチのgspmsインスタンス生成
# 予測モデル: lsvc_gs（線形サポートベクターマシン識別機）
# 調整するパラメータ: tuning_params
# 交差確認法における分割ブロック数: cv
# スコアリング方法: f1値、マイクロ平均
# 並列処理数: n_jobs（1なら並列処理なし、-1なら全てのプロセッサ使用）
# テストデータのサンプル数に基づくスコアへの重み付け: iid（将来、このオプションは削除される）
# 学習データのスコア出力: return_train_score（現状のdefaultは'warm'だが、将来このオプションはFalseになる）
gs_lsvc = GridSearchCV(
    lsvc_gs, tuning_params,
    cv=10, scoring='f1_micro', n_jobs=-1, 
    iid=False, return_train_score=False
)

In [ ]:
# グリッドサーチの実施

gs_lsvc.fit(st_X_train, y_train)

In [ ]:
# 交差検証の結果（cv_results_属性）をDataFrameとして表示
# 平均精度を降順に並べ替え

lsvc_gs_results = pd.DataFrame(gs_lsvc.cv_results_)
lsvc_gs_results.sort_values('mean_test_score', ascending=False)

In [ ]:
print('gs_resultsのサイズ: ' + str(lsvc_gs_results.shape) )
print('最良平均精度: ' + str(lsvc_gs_results.loc[lsvc_gs_results.index[0], 'mean_test_score']))

In [ ]:
# 最適パラメータの確認（best_estimator_属性）

# 交差検証の結果、正則化指標'penalty'はL2ノルム'、
# 正則化の強度'C'は10の1.5乗、損失関数'loss'は'ヒンジ損失'
# の組み合わせが最適であるという結果を得る

gs_lsvc.best_estimator_

In [ ]:
# 判別超平面の係数の可視化

# DataFrameに格納
coefs = pd.DataFrame(
    gs_lsvc.best_estimator_.coef_,
    columns=df.columns[:-1]
)
coefs['intercept'] = gs_lsvc.best_estimator_.intercept_

# DataFrameの可視化
coefs.T.plot(y=coefs.index, kind='bar', title='coefficients', grid=True, figsize=(12,9))

In [ ]:
# 予測値の取得

y_pred = gs_lsvc.predict(st_X_test)

In [ ]:
# 評価指標の出力

# classification_reportの出力
# 結果として、グリッドサーチの最良平均精度は、ロジスティック回帰モデルのものよりも高いが、
# 予測精度としては若干高い程度であった
print(classification_report(y_test, y_pred, labels=labels))

# 混同行列の可視化
conf_mtx = confusion_matrix(y_test, y_pred, labels=labels)
sns.heatmap(conf_mtx, annot=True, xticklabels=labels, yticklabels=labels)

## 補足

今回利用した主なライブラリの説明は以下の通りです  
http://www.numpy.org/  
https://pandas.pydata.org/  
https://matplotlib.org/  
https://seaborn.pydata.org/  
http://scikit-learn.org/stable/  

今回利用したデータは以下からダウンロード可能  
https://github.com/niharikagulati/diabetesprediction/blob/master/diabetes.csv

上記の分析には、異常値の除外や非線形予測モデルの適用はしておりません。  
これらの対応によって精度が向上するか、検証してみてください。